In [1]:
! pip install pytorch-tabnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.8 MB/s eta 0:00:00


In [29]:
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.8 MB/s eta 0:00:00


In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [24]:
match_data = pd.read_csv('/content/drive/MyDrive/nextorial/data/match_data.csv')
test_data = pd.read_csv('/content/drive/MyDrive/nextorial/data/test_data.csv')

In [25]:
class DataProcessor:

    def __init__(self, df):
        self.data = df

    @staticmethod
    # def convert_tier_to_numeric(tier):
    #     tier_dict = {
    #         'unranked': 0,
    #         'bronze': 1,
    #         'silver': 2,
    #         'gold': 3,
    #         'platinum': 4,
    #         'diamond': 5,
    #         'master': 6
    #     }
    #     return tier_dict.get(tier, -1)

    @staticmethod
    def convert_tier_to_exponential_weight(tier):
        tier_dict = {
            'unranked': 0,
            'bronze': 1,
            'silver': 2,
            'gold': 3,
            'platinum': 4,
            'diamond': 5,
            'master': 6
        }
        tier_numeric = tier_dict.get(tier, -1)
        return np.exp(tier_numeric)

    def add_tier_exponential_weight(self):
        self.data['tier_exp_weight'] = self.data['tier'].apply(self.convert_tier_to_exponential_weight)
        return self.data

    # def calculate_team_features(self):
    #     return self.data.groupby(['matchid', 'teamid']).agg({
    #         'mmr': 'mean',
    #         'winstreak': 'mean',
    #         'losestreak': 'mean',
    #         'recentwinprob': 'mean'
    #     }).reset_index()

    def preprocess(self):
        # match_team_features = self.calculate_team_features()
        # self.data = self.data.merge(match_team_features, on=['matchid', 'teamid'], suffixes=('', '_team_avg'))
        self.data = self.normalize_column('accumatches')
        self.data = self.compute_team_stats()
        self.data = self.compute_recent_performance_index()
        self.data = self.process_guild_info(202068.571428571428400)
        self.data = self.compute_mmr_diff_and_variance()
        self.data = self.compute_recent_winprob_stats()
        self.data = self.apply_tier_conversion_and_compute_average()
        self.data = self.compute_streak_rate()
        self.data = self.guild_mean()
        self.data = self.guild_median()
        self.data = self.guild_mode()
        self.data = self.add_tier_exponential_weight()

    # guild_membership
    def guild_mean(self):
        df = self.data.copy()
        df['guildid'].fillna('NoGuild', inplace=True)
        df['tier_numeric'] = df['tier'].map(self.convert_tier_to_exponential_weight)
        guild_mean = df.groupby('guildid')[['mmr', 'winstreak', 'recentwinprob', 'accumatches', 'tier_numeric']].mean()
        new_columns = {col: f'{col}guild_mean' for col in guild_mean.columns}
        guild_mean.rename(columns=new_columns, inplace=True)
        df = pd.merge(df, guild_mean, on='guildid', how='left')
        return df

    def guild_median(self):
        df = self.data.copy()
        df['guildid'].fillna('NoGuild', inplace=True)
        df['tier_numeric'] = df['tier'].map(self.convert_tier_to_exponential_weight)
        guild_median = df.groupby('guildid')[['mmr', 'winstreak', 'recentwinprob', 'accumatches', 'tier_numeric']].median()
        new_columns = {col: f'{col}guild_median' for col in guild_median.columns}
        guild_median.rename(columns=new_columns, inplace=True)
        df = pd.merge(df, guild_median, on='guildid', how='left')
        return df

    def guild_mode(self):
        df = self.data.copy()
        df['guildid'].fillna('NoGuild', inplace=True)
        df['tier_numeric'] = df['tier'].map(self.convert_tier_to_exponential_weight)

        def calculate_mode(group):
            return group.mode().iloc[0]

        guild_mode = df.groupby('guildid')[['mmr', 'winstreak', 'recentwinprob', 'accumatches', 'tier_numeric']].apply(calculate_mode)
        new_columns = {col: f'{col}guild_mode' for col in guild_mode.columns}
        guild_mode.rename(columns=new_columns, inplace=True)
        df = pd.merge(df, guild_mode, on='guildid', how='left')
        return df

    def normalize_column(self, column):
        self.data[f'normalized_{column}'] = (self.data[column] - self.data[column].min()) / (self.data[column].max() - self.data[column].min())
        return self.data

    def compute_team_stats(self):
        grouped = self.data.groupby(['matchid', 'teamid'])
        self.data['team_max_accumatches'] = grouped['accumatches'].transform('max')
        self.data['team_min_accumatches'] = grouped['accumatches'].transform('min')
        self.data['accumatches_diff'] = self.data['team_max_accumatches'] - self.data['team_min_accumatches']
        self.data['accumatches_variance'] = grouped['accumatches'].transform('var')
        return self.data

    def compute_recent_performance_index(self):
        self.data['recent_performance_index'] = self.data['winstreak'] * self.data['recentwinprob']
        return self.data

    def process_guild_info(self, threshold):
        guild_mean_mmr = self.data.groupby('guildid')['mmr'].mean()
        self.data['guild_mean_mmr'] = self.data['guildid'].map(guild_mean_mmr)
        self.data['high_mmr_guild'] = (self.data['guild_mean_mmr'] > threshold).astype(int)
        return self.data

    def compute_mmr_diff_and_variance(self):
        mmr_diff_grouped = self.data.groupby('teamid')['mmr'].agg(['max', 'min'])
        self.data['mmr_diff'] = self.data['teamid'].map(mmr_diff_grouped['max'] - mmr_diff_grouped['min'])
        mmr_variance_grouped = self.data.groupby('teamid')['mmr'].var()
        self.data['mmr_variance'] = self.data['teamid'].map(mmr_variance_grouped)
        return self.data

    def compute_recent_winprob_stats(self):
        grouped = self.data.groupby('matchid')
        self.data['recentwinprob_max'] = grouped['recentwinprob'].transform('max')
        self.data['recentwinprob_min'] = grouped['recentwinprob'].transform('min')
        self.data['recentwinprob_diff'] = self.data['recentwinprob_max'] - self.data['recentwinprob_min']
        self.data['recentwinprob_mean'] = grouped['recentwinprob'].transform('mean')
        self.data['recentwinprob_diff_from_mean'] = (self.data['recentwinprob'] - self.data['recentwinprob_mean'])**2
        self.data['recentwinprob_variance'] = grouped['recentwinprob_diff_from_mean'].transform('mean')
        return self.data

    def apply_tier_conversion_and_compute_average(self):
        self.data['tier_numeric'] = self.data['tier'].apply(self.convert_tier_to_exponential_weight)
        average_tier = self.data.groupby(['matchid', 'teamid'])['tier_numeric'].mean().reset_index()
        average_tier.rename(columns={'tier_numeric': 'average_tier'}, inplace=True)
        self.data = self.data.merge(average_tier, on=['matchid', 'teamid'])
        return self.data

    @staticmethod
    def calculate_streak_rate(row):
        winstreak, losestreak = row['winstreak'], row['losestreak']
        if winstreak + losestreak == 0:
            return 0
        return winstreak / (winstreak + losestreak)

    def compute_streak_rate(self):
        self.data['streak_rate'] = self.data.apply(self.calculate_streak_rate, axis=1)
        return self.data

In [26]:
processor = DataProcessor(match_data)
processor.preprocess()
processed_data = processor.data

In [17]:
# # Processed data is already defined in the context, we'll just export it to a CSV file
# processed_data.to_csv('/content/drive/MyDrive/nextorial/data/processed_match_data.csv', index=False)

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# 데이터에서 특성과 타겟 변수를 분리합니다.
target = 'matchresult'
# Drop identifiers and the original 'tier' feature
features_to_drop = ['createdatekst', 'matchid', 'accountid', 'guildid', 'tier', 'matchscore', 'isDrop', 'isEscape']
features = processed_data.columns.drop([target] + features_to_drop)
X = processed_data[features]
y = processed_data[target]

# 수치형 및 범주형 데이터를 위한 변환기 정의
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# 순서형 특성은 이미 'tier_numeric'으로 변환되어 있으므로 여기서는 처리하지 않습니다.

# 수치형 및 범주형 데이터에 대한 파이프라인을 만듭니다.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))  # NaN 값을 중앙값으로 대체
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # NaN 값을 최빈값으로 대체
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # 원-핫 인코딩
])

# ColumnTransformer를 생성하여 변환기를 결합합니다.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'  # 나머지 열은 변경하지 않고 유지
)

# 전처리를 특성 데이터에 적용합니다.
X_encoded = preprocessor.fit_transform(X)

# 데이터셋을 훈련 세트와 검증/테스트 세트로 분할합니다.
X_temp, X_test, y_temp, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2


In [19]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [20]:
# from pytorch_tabnet.tab_model import TabNetClassifier

# clf = TabNetClassifier()
# clf.fit(
#     X_train, y_train,
#     eval_set=[(X_valid, y_valid)],
#     batch_size = 8,
#     max_epochs = 2,
# )
# preds = clf.predict(X_test)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.68641 | val_0_auc: 0.58574 |  0:08:25s
epoch 1  | loss: 0.68401 | val_0_auc: 0.58654 |  0:17:22s
Stop training because you reached max_epochs = 2 with best_epoch = 1 and best_val_0_auc = 0.58654


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [23]:
# from pytorch_tabnet.tab_model import TabNetClassifier

# clf = TabNetClassifier()
# clf.fit(
#     X_train, y_train,
#     eval_set=[(X_valid, y_valid)],
#     batch_size = 1024,
#     max_epochs = 50,
# )
# preds = clf.predict(X_test)

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.68944 | val_0_auc: 0.57001 |  0:00:12s
epoch 1  | loss: 0.67928 | val_0_auc: 0.59253 |  0:00:24s
epoch 2  | loss: 0.678   | val_0_auc: 0.59655 |  0:00:36s
epoch 3  | loss: 0.67802 | val_0_auc: 0.59728 |  0:00:48s
epoch 4  | loss: 0.67714 | val_0_auc: 0.59968 |  0:01:00s
epoch 5  | loss: 0.67629 | val_0_auc: 0.60159 |  0:01:12s
epoch 6  | loss: 0.67559 | val_0_auc: 0.60327 |  0:01:24s
epoch 7  | loss: 0.67462 | val_0_auc: 0.60496 |  0:01:36s
epoch 8  | loss: 0.67441 | val_0_auc: 0.60629 |  0:01:48s
epoch 9  | loss: 0.67373 | val_0_auc: 0.60862 |  0:02:00s
epoch 10 | loss: 0.67488 | val_0_auc: 0.5941  |  0:02:13s
epoch 11 | loss: 0.6761  | val_0_auc: 0.60057 |  0:02:25s
epoch 12 | loss: 0.67553 | val_0_auc: 0.6047  |  0:02:37s
epoch 13 | loss: 0.67401 | val_0_auc: 0.60788 |  0:02:49s
epoch 14 | loss: 0.67305 | val_0_auc: 0.60875 |  0:03:01s
epoch 15 | loss: 0.67302 | val_0_auc: 0.60912 |  0:03:13s
epoch 16 | loss: 0.67268 | val_0_auc: 0.61012 |  0:03:26s
epoch 17 | los

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [30]:
import optuna
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score

# Optuna를 위한 목적 함수 정의
def objective(trial):

    # 탐색할 하이퍼파라미터 설정
    param = {
        'max_epochs': trial.suggest_int('max_epochs', 10, 30),
        'batch_size': trial.suggest_categorical('batch_size', [256, 512, 1024]),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)
    }

    # 모델 생성
    model = TabNetClassifier(optimizer_params=dict(lr=param['learning_rate']))

    # 모델 학습
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_name=['valid'],
        eval_metric=['accuracy'],
        max_epochs=param['max_epochs'],
        batch_size=param['batch_size'],
        patience=5,
        virtual_batch_size=128,
        num_workers=0,
        drop_last=False
    )

    # 검증 데이터에 대한 예측 및 정확도 측정
    preds = model.predict(X_valid)
    acc = accuracy_score(y_valid, preds)
    return acc

# Optuna study 객체 생성
study = optuna.create_study(direction='maximize', study_name='TabNet optimization')

# 목적 함수에 대한 최적화 실행
study.optimize(objective, n_trials=20)  # n_trials는 시도할 횟수입니다.

# 최적의 하이퍼파라미터 출력
print(f"Best trial: {study.best_trial.params}")
print(f"Best validation accuracy: {study.best_trial.value}")

# 최적의 하이퍼파라미터를 사용하여 최종 모델 학습
best_params = study.best_trial.params
best_model = TabNetClassifier(optimizer_params=dict(lr=best_params['learning_rate']))

best_model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    max_epochs=best_params['max_epochs'],
    batch_size=best_params['batch_size'],
    patience=10,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

# 테스트 데이터에 대한 최종 평가
final_preds = best_model.predict(X_test)
final_acc = accuracy_score(y_test, final_preds)
print(f"Test accuracy: {final_acc}")


[I 2023-10-29 18:09:13,652] A new study created in memory with name: TabNet optimization
<ipython-input-15-11ea38e9d19a>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.69429 | valid_accuracy: 0.50973 |  0:00:12s
epoch 1  | loss: 0.69235 | valid_accuracy: 0.5007  |  0:00:23s
epoch 2  | loss: 0.69188 | valid_accuracy: 0.50082 |  0:00:35s
epoch 3  | loss: 0.6913  | valid_accuracy: 0.50074 |  0:00:47s
epoch 4  | loss: 0.69128 | valid_accuracy: 0.50071 |  0:00:59s
epoch 5  | loss: 0.69128 | valid_accuracy: 0.50071 |  0:01:11s

Early stopping occurred at epoch 5 with best_epoch = 0 and best_valid_accuracy = 0.50973


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-10-29 18:10:50,066] Trial 0 finished with value: 0.5097286694201786 and parameters: {'max_epochs': 11, 'batch_size': 512, 'learning_rate': 0.0750569251240507}. Best is trial 0 with value: 0.5097286694201786.
<ipython-input-15-11ea38e9d19a>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.69854 | valid_accuracy: 0.504   |  0:00:06s
epoch 1  | loss: 0.6907  | valid_accuracy: 0.5201  |  0:00:13s
epoch 2  | loss: 0.68954 | valid_accuracy: 0.53061 |  0:00:20s
epoch 3  | loss: 0.68866 | valid_accuracy: 0.53362 |  0:00:27s
epoch 4  | loss: 0.68848 | valid_accuracy: 0.53405 |  0:00:34s
epoch 5  | loss: 0.68821 | valid_accuracy: 0.53061 |  0:00:41s
epoch 6  | loss: 0.68813 | valid_accuracy: 0.53153 |  0:00:48s
epoch 7  | loss: 0.68795 | valid_accuracy: 0.53183 |  0:00:55s
epoch 8  | loss: 0.68749 | valid_accuracy: 0.53319 |  0:01:02s
epoch 9  | loss: 0.68727 | valid_accuracy: 0.53207 |  0:01:09s

Early stopping occurred at epoch 9 with best_epoch = 4 and best_valid_accuracy = 0.53405


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-10-29 18:12:12,766] Trial 1 finished with value: 0.5340472970503308 and parameters: {'max_epochs': 26, 'batch_size': 1024, 'learning_rate': 0.018006814304556498}. Best is trial 1 with value: 0.5340472970503308.
<ipython-input-15-11ea38e9d19a>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


...

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-10-29 19:00:56,758] Trial 15 finished with value: 0.5479610609549569 and parameters: {'max_epochs': 30, 'batch_size': 512, 'learning_rate': 0.008836208769242107}. Best is trial 15 with value: 0.5479610609549569.
<ipython-input-15-11ea38e9d19a>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.70252 | valid_accuracy: 0.51286 |  0:00:11s
epoch 1  | loss: 0.69165 | valid_accuracy: 0.5234  |  0:00:23s
epoch 2  | loss: 0.69062 | valid_accuracy: 0.52588 |  0:00:35s
epoch 3  | loss: 0.69005 | valid_accuracy: 0.52721 |  0:00:47s
epoch 4  | loss: 0.68923 | valid_accuracy: 0.52698 |  0:00:59s
epoch 5  | loss: 0.68876 | valid_accuracy: 0.52999 |  0:01:11s
epoch 6  | loss: 0.68804 | valid_accuracy: 0.52961 |  0:01:23s
epoch 7  | loss: 0.68821 | valid_accuracy: 0.53033 |  0:01:35s
epoch 8  | loss: 0.6877  | valid_accuracy: 0.53345 |  0:01:46s
epoch 9  | loss: 0.68728 | valid_accuracy: 0.53438 |  0:01:58s
epoch 10 | loss: 0.68733 | valid_accuracy: 0.53492 |  0:02:10s
epoch 11 | loss: 0.68712 | valid_accuracy: 0.53073 |  0:02:22s
epoch 12 | loss: 0.6873  | valid_accuracy: 0.5311  |  0:02:34s
epoch 13 | loss: 0.68662 | valid_accuracy: 0.53385 |  0:02:46s
epoch 14 | loss: 0.68646 | valid_accuracy: 0.53644 |  0:02:58s
epoch 15 | loss: 0.68614 | valid_accuracy: 0.53461 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-10-29 19:05:19,127] Trial 16 finished with value: 0.5364352985611073 and parameters: {'max_epochs': 30, 'batch_size': 512, 'learning_rate': 0.006512395132047262}. Best is trial 15 with value: 0.5479610609549569.
<ipython-input-15-11ea38e9d19a>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.70467 | valid_accuracy: 0.50945 |  0:00:11s
epoch 1  | loss: 0.69171 | valid_accuracy: 0.52404 |  0:00:23s
epoch 2  | loss: 0.6905  | valid_accuracy: 0.52442 |  0:00:35s
epoch 3  | loss: 0.69011 | valid_accuracy: 0.52702 |  0:00:47s
epoch 4  | loss: 0.68947 | valid_accuracy: 0.5283  |  0:00:59s
epoch 5  | loss: 0.68878 | valid_accuracy: 0.52824 |  0:01:11s
epoch 6  | loss: 0.68819 | valid_accuracy: 0.53009 |  0:01:23s
epoch 7  | loss: 0.68775 | valid_accuracy: 0.534   |  0:01:35s
epoch 8  | loss: 0.68727 | valid_accuracy: 0.53478 |  0:01:47s
epoch 9  | loss: 0.68702 | valid_accuracy: 0.53631 |  0:01:59s
epoch 10 | loss: 0.68705 | valid_accuracy: 0.53668 |  0:02:11s
epoch 11 | loss: 0.6867  | valid_accuracy: 0.53901 |  0:02:23s
epoch 12 | loss: 0.68636 | valid_accuracy: 0.53882 |  0:02:34s
epoch 13 | loss: 0.68618 | valid_accuracy: 0.53715 |  0:02:46s
epoch 14 | loss: 0.68623 | valid_accuracy: 0.53765 |  0:02:58s
epoch 15 | loss: 0.68642 | valid_accuracy: 0.53835 |  0

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2023-10-29 19:09:06,644] Trial 17 finished with value: 0.5390060552895451 and parameters: {'max_epochs': 27, 'batch_size': 512, 'learning_rate': 0.005136343147708747}. Best is trial 15 with value: 0.5479610609549569.
<ipython-input-15-11ea38e9d19a>:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)
/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")
